# EEG

In [1]:
import os, sys
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import plotly.express as px

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Importação dos arquivos do bucket

### Verificação de importação dos arquivos

In [2]:
# TROCAR CAMINHO DE IMPORTAÇÃO
files_path = './dataset_eeg_cafe2022/RenamedFiles'
files = [file for file in os.listdir(files_path) if '.csv' in file]

if len(files) == 0:
    print('Sem dados no diretório especificado.')
    exit()

#### Visualização de um sample de dados

In [3]:
df = pd.read_csv(f'{files_path}/{files[0]}', delimiter=',')
df.head()

,Unnamed: 0,Index,Fp1,Fp2,C3,C4,P7,P8,O1,O2,Timestamp
0,0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
1,1,1.0,-133733.943635,-131539.086450,-6371.096917,-144063.160631,54999.961288,-131700.485978,47753.818260,38485.722313,0.002993
2,2,2.0,-49811.172689,-46441.766220,-55496.160431,-57571.708973,521.059546,-98333.738695,-20709.829539,-32717.961654,0.007032
3,3,3.0,-105017.499957,-105803.902413,-33016.389667,-104240.338814,30073.062806,-119989.941209,12879.836209,1598.803011,0.010051
4,4,4.0,-191035.949664,-191928.689828,40018.876501,-193356.172378,84012.637598,-163045.456747,76833.995083,70555.976049,0.013991


### Canais a serem utilizados

In [4]:
channels = df.columns.to_list()[2:10]
channels

['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

### Plotando os sinais

In [5]:
window_widget = widgets.IntSlider(min=1, max=1000, step=10, value=50)
file_widget = widgets.Combobox(options=files, value=files[0], description='Arquivo')
channel_widget = widgets.Dropdown(options=channels, value=channels[0], description='Canal')

In [6]:
def plot_signal(file, channel, window):
    delta = 30
    df = pd.read_csv(f'{files_path}/{file}', delimiter=',')
    timestamps = df['Timestamp'].to_numpy()
    channel_data = df[channel].to_numpy()
    moving_average = np.convolve(channel_data, np.ones(512)/512, mode='valid')
    first_derivative = 100*(moving_average[:-delta] - moving_average[delta:])/delta
    fig = px.line(x=timestamps[:len(first_derivative)], y=[channel_data[:len(first_derivative)], moving_average[:len(first_derivative)], first_derivative])
    fig.update_yaxes(range=[moving_average[10]-5E3, moving_average[10]+5E3])
    fig.show()

In [7]:
out = widgets.interactive_output(plot_signal, 
                                 {'file': file_widget, 
                                  'channel': channel_widget, 
                                  'window': window_widget})

widgets.VBox([widgets.HBox([file_widget, channel_widget, window_widget]), out])